In [1]:
#import re
#import urllib.request

In [99]:
#response = urllib.request.urlopen('https://aip.com.au/pricing/new-south-walesact-retail-petrol-prices')
#html = response.read()
#text = html.decode()

#print (text)

In [107]:
from selenium import webdriver
import re # Importing regex
import datetime
import csv

In [2]:
# Driver is a necessary program that is used to emulate a web browser activity allowing us to view dynamic content
# As the site that we are targeting uses dynamic content, we can only view it via this
# The path to the web driver is relative to the program, hence just place this program and the webdriver in the same folder
path = r'.\Chromedriver'
driver = webdriver.Chrome(executable_path = path)

In [3]:
# Executing the driver with the web address we're targeting
driver.get('https://aip.com.au/pricing/new-south-walesact-retail-petrol-prices')

In [4]:

def split_string(fstring):
    # Return is string is empty
    if len(fstring) <= 0:
        return []
    
    # Check if current char is not a white space
    if fstring[:1] != ' ':
        # Current value found, get word remainder
        value = split_word(fstring)
        
        # Return array of found value and remainder
        return [value[0]] + split_string(value[1])
        
    else:
        # No current value, continue to next
        return split_string(fstring[1:])
    
    # If current char is whitespace but next is a letter, then complete sentence
    #elif fstring[1:][:1].isalpha():
    

def split_word(fstring):
    # Check if we've got chars left
    if len(fstring) <= 0:
        return '', ''
    
    # Check if we haven't reached the end of the word
    if (fstring[:1] != ' ') | ((len(fstring) >= 2) & (fstring[1:2].isalpha())):
        value = split_word(fstring[1:])
        return fstring[:1] + value[0] , value[1]
    
    else:
        return '', fstring
    
    return '', ''



In [14]:
def get_Canberra(flist):
    for element in flist:
        if 'Canberra' in element:
            return element

In [87]:
#test = "Week ending Sunday, 13th September 2020"
#test = "13th September 2020"

#def get_date(fstring):
    #Week ending Sunday, 13th September 2020 Date example
    #ddxx
    #regex = re.search(r"\b[1-3]?[0-9]{1}((th)|(rd)|(st)) [A-Z][a-z]+ [0-9]{4}", fstring)
#    regex = re.search(r"\b[1-3]?[0-9]{1}.* [A-Z][a-z]+ [0-9]{4}", fstring)
#    return regex.group().sub("","((th)|(rd)|(st))") if regex else "None"

#%d %B %Y
#print (get_date(test))

In [27]:
# Driver.find_elements_by_css_selector returns an Array of elements that match the selector's criteria
# We have this in an array which is populated by a for loop which grabs the text values for the elements
#elements = [element.text.split(' ') for element in driver.find_elements_by_css_selector('tr')]
elements = [split_string(element.text) for element in driver.find_elements_by_css_selector('tr')]

# Want to get the current week
#current_week = " ".join(elements[0])

# Will work on getting this single lined
current_prices = []
for element in elements:
    if 'Canberra' in element:
        current_prices = element
        break

Week ending Sunday, 13th September 2020


In [105]:
# Getting date of last Sunday which represents the end of the last week period in order to record accurate data
curr_day = datetime.date.today()
idx = (curr_day.weekday() + 1) % 7
curr_week = curr_day - datetime.timedelta(idx)

TypeError: 'module' object is not callable

In [22]:
driver.close()